In [1]:
import time
import requests
from bs4 import BeautifulSoup
import json
import re
import time
import faker
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from dateutil import parser

In [2]:
fake = faker.Faker(locale='en')

In [3]:
microsoft_headers = {
    'User-Agent': fake.chrome(),
    'accept-language': 'en-US,en;q=0.9',
    'pragma': 'np-cache',
    'host': 'careers.microsoft.com',
    'referer': 'http://careers.microsoft.com/us/en'
}

In [4]:
SEARCH_URL = "http://careers.microsoft.com/us/en/search-results?s=1"
JOB_URL = "http://careers.microsoft.com/us/en/job/"

In [5]:
pattern = re.compile(r'(\{\"jobs\".*).\"aggregations\"', re.MULTILINE)

In [6]:
clear_string = lambda x: re.sub(' +', ' ', re.sub('<.*?>', ' ', x).replace('\n', '\n ')).strip()

In [7]:
job_dicts = []
null_counter = 0
for shift in tqdm(range(0, 10_000, 50)):
    search_params = {
        'from': shift,
    }
    result = requests.get(SEARCH_URL, headers=microsoft_headers, params=search_params)
    soup = BeautifulSoup(result.text)

    found = re.search(pattern, soup.find('script').string).group(1) + "}"
    found = json.loads(found)

    if len(found['jobs']) == 0:
        null_counter += 1
        if null_counter == 3:
            break
    else:
        null_counter = 0

    for job_info in found['jobs']:

        try:
            job_dict = {
                'title': job_info['title'],
                'internal_id' : job_info['jobId'],
                'url': JOB_URL + job_info['jobId'],
                'description': clear_string(job_info['description']),
                'responsibilities': clear_string(job_info['jobResponsibilities']) if 'jobResponsibilities' in job_info.keys() else None,
                'qualifications': clear_string(job_info['jobQualifications']) if 'jobQualifications' in job_info.keys() else None,
                'company': job_info['companyName'],
                'publish_date': parser.parse(job_info['postedDate'])
            }
            job_dicts.append(job_dict)
        except:
            print(f'Cannot parse {JOB_URL + job_info["jobId"]}')

    time.sleep(0.2) # sleep for 200 ms

 52%|█████▎    | 105/200 [05:49<05:16,  3.33s/it]


In [10]:
snapshot = pd.DataFrame(filter(lambda x: x is not None, job_dicts))
snapshot.sample(5)

,title,internal_id,url,description,responsibilities,qualifications,company,publish_date
4342,Senior Software Engineer,1332922,http://careers.microsoft.com/us/en/job/1332922,"The space race is back on, and Microsoft is in...",You will be part of a new development team we ...,&bull; 4+ years Software development experienc...,Microsoft,2022-05-05 19:50:00
1251,Enterprise Digital Sales Specialist - Modern W...,1417789,http://careers.microsoft.com/us/en/job/1417789,The Digital Cloud Acquisition team drives grow...,&nbsp; Responsibilities: Customer First Mindse...,&nbsp; Qualifications 6+ years sales and negot...,Microsoft,2022-07-28 14:33:00
4387,Software Engineer II,1330899,http://careers.microsoft.com/us/en/job/1330899,Microsoft Cloud Operations + Innovation (CO+I)...,Our Systems Engineering team delivers innovati...,"Knowledge, experience, and skills: 4+ years of...",Microsoft,2022-05-02 23:29:00
3736,Senior Technology Architect - Modern Work (Mic...,1358399,http://careers.microsoft.com/us/en/job/1358399,Microsoft is on a mission to empower every per...,Primary accountabilities for this role include...,&nbsp; Growth Mindset. Experience and passion ...,Microsoft,2022-06-12 10:44:00
3733,Account Sales -- Digital Native,1367428,http://careers.microsoft.com/us/en/job/1367428,"This position comes from Web3 ATU, as a GCR ne...",Customer Engagement Proactively cultivates rel...,10+ years experience working in public cloud i...,Microsoft,2022-06-13 05:17:00


In [11]:
snapshot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5124 entries, 0 to 5123
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   title             5124 non-null   object        
 1   internal_id       5124 non-null   object        
 2   url               5124 non-null   object        
 3   description       5124 non-null   object        
 4   responsibilities  5118 non-null   object        
 5   qualifications    5123 non-null   object        
 6   company           5124 non-null   object        
 7   publish_date      5124 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(7)
memory usage: 320.4+ KB


In [12]:
current_date = datetime.now().strftime('%d-%m-%Y')
current_date

'10-08-2022'

In [13]:
snapshot.to_csv(f'../data/microsoft/{current_date}.csv')
snapshot.to_excel(f'../data/microsoft/{current_date}.xlsx', engine='xlsxwriter')

#### Проверка на единичной вакансии

In [7]:
search_params = {
    'from': 0,
}

In [8]:
result = requests.get(SEARCH_URL, headers=microsoft_headers, params=search_params)
result

<Response [200]>

In [9]:
soup = BeautifulSoup(result.text)

In [10]:
soup.find('script').string

' /*<!--*/ var phApp = phApp || {"widgetApiEndpoint":"https://careers.microsoft.com/widgets","country":"us","deviceType":"desktop","locale":"en_us","absUrl":true,"refNum":"MICRUS","cdnUrl":"https://prodcmscdn.azureedge.net/careerconnectresources/p","baseUrl":"https://careers.microsoft.com/us/en/","baseDomain":"https://careers.microsoft.com","phenomTrackURL":"careers.microsoft.com/us/en/phenomtrack.min.js","pageName":"search-results","siteType":"external","rootDomain":"https://careers.microsoft.com","pageId":"page19"}; phApp.ddo = {"siteConfig":{"status":"success","errorCode":null,"errorMsg":null,"data":{"urlMap":{"home":"home","category":"c/:category","job":"job/:jobSeqNo/:title","jobcart":"jobcart","search-results":"search-results","glassdoor-reviews":"glassdoor-reviews"},"categoryUrlMap":{"MICRUS_Legal---Corporate-Affairs":"c/legal-corporate-affairs-jobs","MICRUS_Supply-Chain---Operations-Management":"c/supply-chain-operations-management-jobs","MICRUS_Field-Business-Leadership":"c/fi

здесь в phApp.ddo есть описания вакансий, но нужено парсить джаваскрипт... конкретнее нас инетерсует ключ eagerLoadRefineSearch внутри оson

In [212]:
# soup.find('script').string

In [11]:
found = re.search(pattern, soup.find('script').string).group(1) + "}"
found = json.loads(found)
len(found['jobs'])

50

In [12]:
[(i, j['title']) for i, j in enumerate(found['jobs'])]

[(0, 'Cloud Solution Architecture Director-Financial Services'),
 (1, 'Technical Support Advisory IC4'),
 (2, 'Support Engineer - Real Time Support (VM)'),
 (3, 'Support Engineer - Real Time Support (Containers)'),
 (4,
  'Enterprise Digital Sales Specialist - Modern Work, Employee Experience - Brazil (Portuguese Speaking)'),
 (5, 'DC Technician II'),
 (6, 'Global Black Belt Specialist, Cross Solutions'),
 (7, 'DC Technician II'),
 (8, 'Technical Support Engineering M6'),
 (9,
  'Corporate Digital Specialist - Modern Work - MEA (Arabic and English Speaking)'),
 (10, 'Global Black Belt Specialists Manager, Cross Solutions'),
 (11, 'GBB Time Zone Leader, Cross Solutions - Asia'),
 (12, 'Global Infra Cloud Solution Architect'),
 (13, 'Digital Technical Specialist - Security & Compliance - German speaker'),
 (14, 'Technical Support Engineering IC3'),
 (15, 'Digital Cloud Solution Architecture - Italian speaker'),
 (16,
  'Digital Sales Specialist - Enterprise - Security - Switzerland (Germ

In [13]:
parser.parse(found['jobs'][0]['postedDate'])

datetime.datetime(2022, 7, 28, 15, 14)

In [14]:
found['jobs'][0].keys()

dict_keys(['country', 'subCategory', 'city', 'companyName', 'description', 'industry', 'title', 'multi_location', 'type', 'orgFunction', 'experience', 'locale', 'multi_location_array', 'jobQualifications', 'jobSeqNo', 'postedDate', 'searchresults_display', 'descriptionTeaser', 'jobResponsibilities', 'dateCreated', 'educationLevel', 'state', 'jd_display', 'employmentType', 'parentRefNum', 'jobSummary', 'reqId', 'badge', 'jobId', 'isMultiLocation', 'jobVisibility', 'refNum', 'mostpopular', 'applyUrl', 'worksite', 'location', 'category', 'locationLatlong', 'requisitionRoleType'])

In [15]:
'state' in found['jobs'][0].keys()

True

In [18]:
print(clear_string(found['jobs'][0]['jobQualifications']))

Qualifications Required/Minimum Qualifications Bachelor's Degree in Computer Science, Information Technology, Engineering, Business, Liberal Arts, or related field AND 8+ years experience in cloud/infrastructure technologies, information technology (IT) consulting/support, systems administration, network operations, software development/support, technology solutions, practice development, architecture, and/or consulting OR equivalent experience. 3+ years people management experience, including managing consultant practice managers, technical sales managers, and/or technical architect managers. &nbsp; Additional or Preferred Qualifications Bachelor's Degree in Computer Science, Information Technology, Engineering, Business, Liberal Arts, or related field AND 12+ years experience in cloud/infrastructure technologies, information technology (IT) consulting/support, systems administration, network operations, software development/support, technology solutions, practice development, archite